# Exploración Inicial

__Descripción:__

Se obtiene el AGEB de origen y destino de los viajes de los viajes de ecobici y se obtiene el agregado primedio de los viajes entre AGEBS por hora.

__Input__

- Estaciones de ecobici: Estaciones de ecobici con clave AGEB obtenida de __Uber © 2019 Copyright Uber Technologies, Inc.Data Attributions__

- Viajes Ecobici 2019
Responsable: Ecobici



In [3]:
import pandas as pd
import pickle

import pandas as pd
import dask.dataframe as dd
import time

import scipy.stats

## Obtencion de AGEBS de interés:

In [4]:
df_estaciones = pd.read_csv("./SIG/data/procesado/CSV/estaciones-de-ecobici_con_AGEB.csv")
df_estaciones.head()

,ID,Nombre,Dirección,Número,Código po,districtCo,Colonia,altitude,nearbyStat,location/l,...,nearbySt_3,nearbySt_4,nearbySt_5,punto_geo,CVE_AGEB,CVE_MUN,CVE_LOC,CVE_ENT,MOVEMENT_I,DISPLAY_NA
0,440,440 REAL DE MAYORAZGO-PUENTE XOCO,440 - Real De Mayorazgo-Puente Xoco,S/N,3330.0,1,Ampliación Granada,NaN,433,19.360650,...,NaN,NaN,NaN,"19.36065,-99.168669",0967,14,1,9,4550,Movement Zone 4550
1,283,283 AV. COYOACÁN-RAFAEL DONDE,283 - Av. Coyoacán-Rafael Donde,S/N,3103.0,1,Ampliación Granada,NaN,279,19.398270,...,NaN,NaN,NaN,"19.39827,-99.167383",0045,14,1,9,4551,Movement Zone 4551
2,279,279 LA MORENA-PROVIDENCIA,279 - La Morena-Providencia,S/N,3103.0,DVN,Del Valle Norte,NaN,164,19.399908,...,NaN,NaN,NaN,"19.399908,-99.166553",0045,14,1,9,4551,Movement Zone 4551
3,443,443 BRUNO TRAVEN-GOLONDRINAS,443 - Bruno Trave-Avenida México-Coyoacan,S/N,3340.0,1,Ampliación Granada,NaN,432,19.359655,...,NaN,NaN,NaN,"19.359655,-99.162025",0971,14,1,9,4553,Movement Zone 4553
4,432,432 CARRILLO PUERTO-AV. MÉXICO-COYOACÁN,432 - Carrillo Pto-Golondrinas,S/N,3340.0,GPM,General Pedro Maria Anaya,NaN,442,19.361296,...,NaN,NaN,NaN,"19.361296,-99.162581",0971,14,1,9,4553,Movement Zone 4553


In [13]:
lista_agebs = df_estaciones['CVE_AGEB'].unique().tolist()
len(lista_agebs)

143

Guardamos lo AGEBS en una lista:

In [14]:
with open("./data/lista_agebs.txt", "wb") as fp:
    pickle.dump(lista_agebs, fp)

## Leamos el archivo de viajes de UBER

In [21]:
path_uber_trips = "./data/uber/mexico_city-agebs-2019-2-All-HourlyAggregate.csv"

In [22]:
t=time.clock()
df_trips = dd.read_csv(path_uber_trips)
df_trips = df_trips.compute()
print("load train: " , time.clock()-t)

load train:  70.403141


In [16]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,3409,2789,11,1593.14,546.69,1529.45,1.30
1,3241,4552,7,2198.52,501.11,2119.54,1.34
2,3185,2672,8,2115.36,457.21,2065.90,1.24
3,201,2147,11,641.64,319.21,591.79,1.44
4,301,1147,11,1632.54,550.32,1566.17,1.31


In [23]:
df_trips = df_trips[df_trips["sourceid"].astype(str).isin(lista_agebs)]

In [25]:
df_trips.to_csv('./data/procesado/mexico_city-agebs-2019-2-All-HourlyAggregate_filter.csv')

In [49]:
df_trips.dtypes

sourceid                                      int64
dstid                                         int64
hod                                           int64
mean_travel_time                            float64
standard_deviation_travel_time              float64
geometric_mean_travel_time                  float64
geometric_standard_deviation_travel_time    float64
dtype: object

In [50]:
df_trips.reset_index(drop=True, inplace=True)

In [51]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,1375,2067,11,2125.00,640.25,2039.71,1.32
1,1534,2452,5,766.98,243.10,743.55,1.28
2,1106,1944,19,1298.45,258.22,1274.28,1.21
3,1303,1894,16,1124.22,278.44,1098.68,1.22
4,1375,1174,16,1222.41,403.44,1173.74,1.31


### tiempo máximo y mínimos por viaje y hora.
Suponemos una distribución normal con media 'mean_travel_time' y desviación estandar 'standard_deviation_travel_time>'

In [80]:
def travel_min(mean,std):
    min = scipy.stats.norm(mean, std).ppf(.10)
    #max = scipy.stats.norm(mean, std).ppf(.90)
    return(min)
def travel_max(mean,std):
    #min = scipy.stats.norm(mean, std).ppf(.10)
    max = scipy.stats.norm(mean, std).ppf(.90)
    return(max)

In [81]:
df_trips['min_travel_time'] = 0
df_trips['max_travel_time'] = 0

In [83]:
df_trips['min_travel_time'] =  df_trips.apply(
    lambda row : travel_min(row['mean_travel_time'], 
                            row['standard_deviation_travel_time']), axis = 1)

In [84]:
df_trips['max_travel_time'] =  df_trips.apply(
    lambda row : travel_max(row['mean_travel_time'], 
                            row['standard_deviation_travel_time']), axis = 1)

In [92]:
df_trips.to_csv('./data/procesado/mexico_city-agebs-2019-2-All-HourlyAggregate_filter.csv')

In [93]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,min_travel_time,max_travel_time
0,1375,2067,11,2125.00,640.25,2039.71,1.32,1304.486610,2945.513390
1,1534,2452,5,766.98,243.10,743.55,1.28,455.434814,1078.525186
2,1106,1944,19,1298.45,258.22,1274.28,1.21,967.527755,1629.372245
3,1303,1894,16,1124.22,278.44,1098.68,1.22,767.384782,1481.055218
4,1375,1174,16,1222.41,403.44,1173.74,1.31,705.380836,1739.439164


In [96]:
df_trips.dtypes

sourceid                                      int64
dstid                                         int64
hod                                           int64
mean_travel_time                            float64
standard_deviation_travel_time              float64
geometric_mean_travel_time                  float64
geometric_standard_deviation_travel_time    float64
min_travel_time                             float64
max_travel_time                             float64
dtype: object

In [13]:
df_viajes_ageb.groupby(['Ciclo_Estacion_Retiro','Ciclo_Estacion_Arribo','Hora_Retiro']).mean().reset_index(drop=False)

,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Hora_Retiro,Edad_Usuario,ID_y,Código po,altitude,nearbyStat,location/l,location_1,nearbySt_1,nearbySt_2,nearbySt_3,nearbySt_4,nearbySt_5,CVE_MUN,CVE_LOC,CVE_ENT,MOVEMENT_I
0,1,1,00:07:50,30.0,1.0,6500.0,NaN,3.0,19.433571,-99.167809,8.0,85.0,NaN,NaN,NaN,15.0,1.0,9.0,4749.0
1,1,1,00:09:41,72.0,1.0,6500.0,NaN,3.0,19.433571,-99.167809,8.0,85.0,NaN,NaN,NaN,15.0,1.0,9.0,4749.0
2,1,1,00:16:05,32.0,1.0,6500.0,NaN,3.0,19.433571,-99.167809,8.0,85.0,NaN,NaN,NaN,15.0,1.0,9.0,4749.0
3,1,1,00:22:41,25.0,1.0,6500.0,NaN,3.0,19.433571,-99.167809,8.0,85.0,NaN,NaN,NaN,15.0,1.0,9.0,4749.0
4,1,1,00:29:13,74.0,1.0,6500.0,NaN,3.0,19.433571,-99.167809,8.0,85.0,NaN,NaN,NaN,15.0,1.0,9.0,4749.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6287823,1002,445,10:22:34,29.0,445.0,3340.0,NaN,431.0,19.358270,-99.156105,434.0,446.0,NaN,NaN,NaN,14.0,1.0,9.0,4569.0
6287824,3000,108,16:14:26,29.0,108.0,6500.0,NaN,33.0,19.427305,-99.149533,98.0,107.0,260.0,NaN,NaN,15.0,1.0,9.0,4754.0
6287825,3000,116,14:32:01,29.0,116.0,6500.0,NaN,115.0,19.428985,-99.156640,118.0,199.0,NaN,NaN,NaN,15.0,1.0,9.0,4752.0
6287826,3000,116,16:18:07,29.0,116.0,6500.0,NaN,115.0,19.428985,-99.156640,118.0,199.0,NaN,NaN,NaN,15.0,1.0,9.0,4752.0
